In [50]:
import onnxruntime

# Define model and data URL
PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

# Download the files
!wget {DATA_URL}
!wget {MODEL_URL}

--2025-12-09 21:31:58--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 4.228.31.150
Connecting to github.com (github.com)|4.228.31.150|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-10T01%3A24%3A44Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-10T00%3A24%3A43Z&ske=2025-12-10T01%3A24%3A44Z&sks=b&skv=2018-11-09&sig=CbZ2s9TBv3q%2Bf4x%2BvSChR1Q97GW51j2trKDPnAjJQbg%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5

In [52]:
# Load the ONNX model and determine the output node name
sess = onnxruntime.InferenceSession("hair_classifier_v1.onnx")
output_name = sess.get_outputs()[0].name
print(f"The output node name is: {output_name}")

The output node name is: output


In [53]:
import onnxruntime
from PIL import Image
from io import BytesIO
from urllib import request
import numpy as np

# Helper functions provided in the homework
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    # The homework uses Image.NEAREST for resizing
    img = img.resize(target_size, Image.NEAREST)
    return img

In [54]:
# Load the ONNX model
sess = onnxruntime.InferenceSession("hair_classifier_v1.onnx")

# Get input and output details from the model
input_details = sess.get_inputs()[0]
output_details = sess.get_outputs()[0]

input_name = input_details.name
input_shape = input_details.shape
output_name = output_details.name

print(f"Model Input Name: {input_name}")
print(f"Model Input Shape: {input_shape}")
print(f"Model Output Name: {output_name}")

Model Input Name: input
Model Input Shape: ['s77', 3, 200, 200]
Model Output Name: output


In [55]:
# Question 2: Determine target image size from model input shape
# Assuming a 4-dimensional input shape (batch, height, width, channels) or (batch, channels, height, width)
if len(input_shape) == 4:
    # Determine if channels are first or last to extract Height and Width
    # Common PyTorch/ONNX models use (N, C, H, W), so C=input_shape[1]
    # Common TensorFlow/Keras models use (N, H, W, C), so C=input_shape[3]
    if input_shape[1] == 3: # Channels first (N, C, H, W)
        target_height = input_shape[2]
        target_width = input_shape[3]
        channels_first = True
    elif input_shape[3] == 3: # Channels last (N, H, W, C)
        target_height = input_shape[1]
        target_width = input_shape[2]
        channels_first = False
    else: # Fallback for unexpected channel counts/order, assume H, W are the next two dims after batch
        target_height = input_shape[1]
        target_width = input_shape[2]
        channels_first = False # Default assumption if channels are not 3
else:
    # Fallback if input shape is not 4D, or if unable to infer. 
    # Based on common practice for similar models, 150x150 is a frequent size.
    target_height = 150
    target_width = 150
    channels_first = False # Default assumption

target_size = (target_width, target_height) # PIL resize expects (width, height)
print(f"\nQuestion 2 - Determined target image size: {target_width}x{target_height}")


Question 2 - Determined target image size: 200x200


In [56]:
# Download and prepare the image
img_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
print(f"Downloading image from: {img_url}")
original_img = download_image(img_url)
prepared_img = prepare_image(original_img, target_size)
print(f"Image downloaded and prepared to size: {prepared_img.size}")

Image downloaded and prepared to size: (200, 200)


In [57]:
# Question 3: Convert to NumPy array, pre-process, and get first pixel R channel
# Convert PIL image to numpy array
x = np.array(prepared_img, dtype=np.float32)

# Adjust shape if channels first is required by the model
if channels_first:
    x = x.transpose(2, 0, 1) # From (H, W, C) to (C, H, W)

# Add batch dimension (model expects batch size 1)
x = np.expand_dims(x, axis=0) # From (C, H, W) to (1, C, H, W) or (H, W, C) to (1, H, W, C)

# Pre-processing: normalize pixel values to [-1, 1]
# (e.g., x / 127.5 - 1.0 for 0-255 pixel range)
x = x / 127.5 - 1.0
print("Image converted to NumPy array and pre-processed for model input.")

# Get the value of the first pixel (top-left, R channel) after pre-processing
# The R channel is the first channel (index 0) regardless of channels_first/last
first_pixel_r_value = x[0, 0, 0, 0]
print(f"Question 3 - Value of the first pixel (R channel) after pre-processing: {first_pixel_r_value}")

Image converted to NumPy array and pre-processed for model input.
Question 3 - Value of the first pixel (R channel) after pre-processing: -0.5215686559677124


In [60]:
# Question 4: Perform model inference
input_feed = {input_name: x}
outputs = sess.run([output_name], input_feed)

# Return the model outputs a single scalar prediction
model_prediction = outputs[0][0][0]
print(f"Question 4 - Model output: {model_prediction}")


Question 4 - Model output: -0.203227236866951


Questions 5 and 6: Create the lambda function file and Dockerfile, then we determine the Docker base image size and perform Docker-based inference.

In [61]:
%%writefile lambda_function.py
import onnxruntime
from PIL import Image
from io import BytesIO
from urllib import request
import numpy as np
import json
import os

def download_image(url):
    """
    Download an image from a given URL and open it as a PIL Image.

    Args:
        url (str): The URL of the image to download.

    Returns:
        PIL.Image: The downloaded image.
    """
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    """
    Prepare the image for model input by converting to RGB and resizing.

    Args:
        img (PIL.Image): Input image.
        target_size (tuple): Desired size as (width, height).

    Returns:
        PIL.Image: Processed image.
    """
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

class ImageClassifier:
    """
    Image classifier using an ONNX model for inference.
    """

    def __init__(self, model_path="hair_classifier_empty.onnx"):
        """
        Initialize the ONNX Runtime session and read model input/output metadata.

        Args:
            model_path (str): Path to the ONNX model file.
        """
        self.sess = onnxruntime.InferenceSession(model_path)
        self.input_name = self.sess.get_inputs()[0].name
        self.input_shape = self.sess.get_inputs()[0].shape
        self.output_name = self.sess.get_outputs()[0].name

        # Determine target input size and channel format (channels first or last)
        if len(self.input_shape) == 4:
            if self.input_shape[1] == 3:  # Channels first (N, C, H, W)
                self.target_height = self.input_shape[2]
                self.target_width = self.input_shape[3]
                self.channels_first = True
            elif self.input_shape[3] == 3:  # Channels last (N, H, W, C)
                self.target_height = self.input_shape[1]
                self.target_width = self.input_shape[2]
                self.channels_first = False
            else:
                raise ValueError("Model input shape does not indicate 3 channels correctly.")
        else:
            raise ValueError("Model input shape is not 4-dimensional.")

        self.target_size = (self.target_width, self.target_height)

    def preprocess(self, img):
        """
        Convert the input PIL image to a normalized numpy tensor suitable for the model.

        Args:
            img (PIL.Image): Preprocessed image of target size.

        Returns:
            np.ndarray: Normalized input tensor with correct shape.
        """
        x = np.array(img, dtype=np.float32)
        # Change shape to channels first if model expects it
        if self.channels_first:
            x = x.transpose(2, 0, 1)
        x = np.expand_dims(x, axis=0)  # Add batch dimension
        x = x / 127.5 - 1.0  # Normalize pixel values to [-1, 1]
        return x

    def predict(self, url):
        """
        Download an image from the URL, preprocess it, and run model inference.

        Args:
            url (str): URL of the image to classify.

        Returns:
            float: Prediction result from the model.
        """
        original_img = download_image(url)
        prepared_img = prepare_image(original_img, self.target_size)
        processed_x = self.preprocess(prepared_img)
        input_feed = {self.input_name: processed_x}
        outputs = self.sess.run([self.output_name], input_feed)
        return outputs[0][0][0]

# Instantiate classifier with the ONNX model file
classifier = ImageClassifier(model_path="hair_classifier_empty.onnx")

def handler(event, context=None):
    """
    AWS Lambda handler function to process incoming event, extract image URL,
    perform prediction, and return response.

    Args:
        event (dict): Event payload containing image URL data.
        context: Lambda context (not used).

    Returns:
        dict: HTTP-like response with prediction or error details.
    """
    url = None

    # Parse URL from event: from JSON body or directly from event keys
    if isinstance(event, dict):
        if 'body' in event and isinstance(event['body'], str):
            try:
                body = json.loads(event['body'])
                url = body.get('url')
            except json.JSONDecodeError:
                pass
        elif 'url' in event:
            url = event.get('url')

    if not url:
        return {
            "statusCode": 400,
            "body": json.dumps({"error": "Invalid event format or image URL not provided"})
        }

    try:
        prediction = classifier.predict(url)
        return {
            "statusCode": 200,
            "body": json.dumps({"prediction": float(prediction)})
        }
    except Exception as e:
        print(f"Error during prediction: {e}")
        return {
            "statusCode": 500,
            "body": json.dumps({"error": str(e)})
        }

Overwriting lambda_function.py


In [62]:
%%writefile Dockerfile
FROM agrigorev/model-2025-hairstyle:v1

RUN pip install --no-cache-dir onnxruntime numpy Pillow==11.3.0

COPY lambda_function.py ${LAMBDA_TASK_ROOT}/

CMD [ "lambda_function.handler" ]

Overwriting Dockerfile


In [63]:
!docker pull agrigorev/model-2025-hairstyle:v1
!docker images agrigorev/model-2025-hairstyle:v1

v1: Pulling from agrigorev/model-2025-hairstyle
Digest: sha256:9e43d5a5323f7f07688c0765d3c0137af66d0154af37833ed721d6b4de6df528
Status: Image is up to date for agrigorev/model-2025-hairstyle:v1
docker.io/agrigorev/model-2025-hairstyle:v1
                                                            i Info →   U  In Use
IMAGE                           ID             DISK USAGE   CONTENT SIZE   EXTRA
agrigorev/model-2025-hairstyle:v1
                                4528ad1525d5        608MB             0B    U   


In [ ]:
# Question 5: Determine the Docker image size
!docker images

In [40]:
# Question 6: Build and run Dockerfile
!docker build -t hair-classifier-lambda .
!docker run -p 8080:8080 hair-classifier-lambda:latest

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  762.6MB
Step 1/4 : FROM agrigorev/model-2025-hairstyle:v1
 ---> 4528ad1525d5
Step 2/4 : RUN pip install --no-cache-dir onnxruntime numpy Pillow==11.3.0
 ---> Using cache
 ---> 415cfe7fe653
Step 3/4 : COPY lambda_function.py ${LAMBDA_TASK_ROOT}/
 ---> Using cache
 ---> c1f7af96301f
Step 4/4 : CMD [ "lambda_function.handler" ]
 ---> Running in 29f3f60a3a80
 ---> Removed intermediate container 29f3f60a3a80
 ---> 84da9517d0eb
Successfully built 84da9517d0eb
Successfully tagged hair-classifier-lambda:latest
10 Dec 2025 00:14:17,741 [INFO] (rapid) exec '/var/runtime/bootstrap' (cwd=/var/task, handler=)
START RequestId: af35da4b-5e93-4c39-885b-6ed7d87d4e97 Version: $LATEST
10 Dec 2025 00:16:34,988 [INFO] (rapid) INIT START(type: on-demand, phase: init

In [65]:
# Model output (docker)
!curl -XPOST "http://localhost:8080/2015-03-31/functions/function/invocations" -d '{"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}'

{"statusCode": 200, "body": "{\"prediction\": -0.09092357009649277}"}